# 7_Pull_avg_hourly_demand_into_csv

In [7]:
%load_ext sql
%load_ext lab_black

In [5]:
from src.features import paths

path_to_data_database = paths.get_path_to_data("data.db")
path_to_data_database

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\processed\\data.db'

In [8]:
%sql sqlite:///$path_to_data_database

'Connected: @C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\processed\\data.db'

In [9]:
%%time
%%sql

SELECT * 
FROM electricity_average_consumption
LIMIT 5;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.
Wall time: 15.6 ms


Date,Time,Consumption
2009-07-15,00:00:00,0.5822443136631121
2009-07-15,00:30:00,0.5020396055352317
2009-07-15,01:00:00,0.4328597105137602
2009-07-15,01:30:00,0.38651121361539587
2009-07-15,02:00:00,0.35428805471608116


# Sum every two rows into an hourly value

In [36]:
%%time
%%sql

SELECT 
    Date, 
    Time, 
    Consumption + COALESCE(LAG(Consumption, 1) OVER (ORDER BY Date, Time), 0) AS Sum_every_two_rows
FROM electricity_average_consumption
LIMIT 6;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.
Wall time: 45 ms


Date,Time,Sum_every_two_rows
2009-07-15,00:00:00,0.5822443136631121
2009-07-15,00:30:00,1.0842839191983438
2009-07-15,01:00:00,0.934899316048992
2009-07-15,01:30:00,0.819370924129156
2009-07-15,02:00:00,0.740799268331477
2009-07-15,02:30:00,0.6844383648799117


In [ ]:
SELECT "date", 
sum("count") OVER (order by "date" ROWS BETWEEN 2 preceding AND current row)
FROM t
ORDER BY "date";

In [41]:
%%time
%%sql

SELECT 
    Date, 
    Time, 
    sum(Consumption) OVER (ORDER BY Date, Time ROWS BETWEEN 1 preceding AND current row)
FROM electricity_average_consumption
LIMIT 6;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.
Wall time: 44.6 ms


Date,Time,"sum(Consumption) OVER (ORDER BY Date, Time ROWS BETWEEN 1 preceding AND current row)"
2009-07-15,00:00:00,0.5822443136631121
2009-07-15,00:30:00,1.0842839191983438
2009-07-15,01:00:00,0.9348993160489918
2009-07-15,01:30:00,0.8193709241291559
2009-07-15,02:00:00,0.7407992683314768
2009-07-15,02:30:00,0.6844383648799115


In [40]:
%%time
%%sql

SELECT 
    Date, 
    Time, 
    CASE
        WHEN 
            Time = LEAD(Time,1,0) OVER (ORDER BY Date, Time) 
        THEN
            Consumption + COALESCE(LAG(Consumption, 1) OVER (ORDER BY Date, Time), 0) 
        ELSE 0
        END Hourly_consumption
FROM electricity_average_consumption
LIMIT 6;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.
Wall time: 321 ms


Date,Time,Hourly_consumption
2009-07-15,00:00:00,0
2009-07-15,00:30:00,0
2009-07-15,01:00:00,0
2009-07-15,01:30:00,0
2009-07-15,02:00:00,0
2009-07-15,02:30:00,0


SOURCES:
- Get sum of every two neighbouring rows https://stackoverflow.com/questions/11268398/sum-every-3-rows-of-a-table
- From second comment of above SO post https://www.postgresql.org/docs/current/sql-expressions.html#SYNTAX-WINDOW-FUNCTIONS

In [38]:
%%time
%%sql

SELECT Date, Time, LEAD(Time,1,0) OVER (ORDER BY Date, Time)
FROM electricity_average_consumption
LIMIT 5;

 * sqlite:///C:\Users\RowanM\Documents\dublin_energy_masterplan\Residential\data\inputs\processed\data.db
Done.
Wall time: 120 ms


Date,Time,"LEAD(Time,1,0) OVER (ORDER BY Date, Time)"
2009-07-15,00:00:00,00:30:00
2009-07-15,00:30:00,01:00:00
2009-07-15,01:00:00,01:30:00
2009-07-15,01:30:00,02:00:00
2009-07-15,02:00:00,02:30:00


SOURCES:
- Use LEAD() to access next row https://www.sqlitetutorial.net/sqlite-window-functions/sqlite-lead/
- Use SUM() HAVING '00' == '00', '01' = '01' etc to sum every hour https://www.sqlitetutorial.net/sqlite-sum/
- Use SUBSTR() To split Time string https://www.sqlitetutorial.net/sqlite-functions/sqlite-substr/

# Push these hourly values into a CSV for use in EnergyPRO

SOURCES:
- Export from database to csv https://www.quackit.com/sqlite/tutorial/export_data_to_csv_file.cfm